In [2]:
# def day4_parser(line: str) -> tuple:
#     pass
    

# def day4_part1(policies: list[tuple]) -> int:
#     pass


# def day4_part2(policies: list[tuple]) -> int:
#     pass

# run(day=3, parser=day4_parser, sep='\n', funcs=[day4_part1, day4_part2], example=True, actual=True)


# for i in range(1, 26):
#     with open(f'data/day{i:02}.txt', 'a') as f:
#         f.write('')

In [56]:
import timeit
from itertools import permutations, combinations, product, chain, islice
from math import ceil, floor

#Dict, Tuple, Set, List, Iterator, Optional, Union, Sequence

# from __future__  import annotations
from collections import defaultdict, namedtuple, deque #Counter, defaultdict, , deque
# from itertools   import permutations, combinations, product, chain
from functools   import partial  #lru_cache, reduce
from typing import Any, Callable, Optional # Iterator, Optional, Union, Sequence, DefaultDict?
# from contextlib  import contextmanager

# import operator
# import math
# import ast
# import sys
import re


from more_itertools import minmax



cat = ''.join


def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)


def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))


def lines(text: str) -> list[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

In [4]:
def data(path: str, parser: Callable[[str], Any]=str, sep: str='\n') -> list:
    "Load input textfile into sections separated by `sep`, and apply `parser` to section." 
    assert isinstance(path, str), path
    with open(path, 'r') as f:
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))


def run(day: int, funcs: list=[], example: bool=True, actual: bool=False, parser=str, sep: str='\n') -> None:
    """Run a day's example (and/or actual) input on a list of callables for each part i.e. [part1, part2]. 
    While working can use [part1, lambda _: None]."""
    if example:
        puzzle_input = data(path=f'data/example/day{day:02}.txt', parser=parser, sep=sep)
        print(f'{"Part 1 (example): " + str(funcs[0](puzzle_input)):<25}', end='\t')
        print(f'{"Part 2 (example): " + str(funcs[1](puzzle_input)):<25}', end='\n')
    
    if actual:
        puzzle_input = data(path=f'data/day{day:02}.txt', parser=parser, sep=sep)
        print(f'{"Part 1 (actual): " + str(funcs[0](puzzle_input)):<25}', end='\t')
        print(f'{"Part 2 (actual): " + str(funcs[1](puzzle_input)):<25}', end='\n')


def test_alternatives(day: int, funcs: list=[], runs: int=10, parser=str, sep: str='\n') -> None:
    "Test alternative approaches on a specific part of a day's actual input."
    puzzle_input = data(path=f'data/day{day:02}.txt', parser=parser, sep=sep)
    
    print('Func Result:', end='\t')
    for func in funcs: 
        print(func(puzzle_input), end='  ')

    print('\nFunc Runtime:', end='\t')
    for func in funcs:  # Compare each functions runtime
        print(f'{timeit.timeit("func(puzzle_input)", number=runs, globals=locals()):.3f}', end='  ')

# Done

## Day 01: Report Repair
- Part 1: find two numbers that sum to 2020, return their product
- Part 2: find three numbers that sum to 2020, return their product

In [4]:
def day1_part1(nums: list[int]) -> int:
    for x in nums:
        for y in nums:
            if x + y == 2020:
                return x * y


def day1_part2(nums: list[int]) -> int:
    for x in nums:
        for y in nums:
            for z in nums:
                if x + y + z == 2020:
                    return x * y * z


run(day=1, parser=int, sep='\n', funcs=[day1_part1, day1_part2], example=False, actual=True)

Part 1 (actual): 802011  	Part 2 (actual): 248607374


### Other Solutions

In [5]:
def day1_part1b(nums: list[int]) -> int: # still O(n^2) as above, but slighty faster
    for i, x in enumerate(nums):
        for y in islice(nums, i): # not using nums[i:] as that creates a copy each time (would make this O(n^3))
            if x + y == 2020:
                return x * y


def day1_part1c(nums: list[int]) -> int: # O(n^2)
    for x, y in combinations(nums, 2): 
        if x + y == 2020:
            return x * y


def day1_part1d(nums: list[int]) -> int: # O(n)
    seen = set()
    for x in nums:
        if (2020 - x) in seen:
            return (2020 - x) * x
        seen.add(x)


def day1_part1e(nums: list[int]) -> int: # O(n) the second loop is only run max twice (i.e. 2020 + 20 and 20 + 2020)
    nums = set(nums) # ideally input would already be a set
    return first(
        x * y 
        for x in nums 
        for y in nums.intersection({2020 - x})
        if x != y
        )


test_alternatives(day=1, funcs=[day1_part1, day1_part1b, day1_part1c, day1_part1d, day1_part1e], runs=1000, parser=int)

Func Result:	802011 802011 802011 802011 802011 
Func Runtime:	0.309  0.145  0.322  0.006  0.003  

In [6]:
def day1_part2b(nums: list[int]) -> int: # still O(n^3) as above, has faster wall time though
    for x, y, z in combinations(nums, 3): 
        if x + y + z == 2020:
            return x * y * z


def day1_part2c(nums: list[int]) -> int: # O(n^2), but slower wall time (on the test input/size)
    set_nums = set(nums)
    for x, y in combinations(nums, 2):
        if (2020 - x - y) in set_nums:
            return x * y * (2020 - x - y)


def day1_part2d(nums: list[int]) -> int: # O(n^2)
    nums = set(nums)
    return first(
        x * y * z 
        for x, y in combinations(nums, 2) 
        for z in nums.intersection({2020 - x - y})
        if x != y != z
        )


def day1_part2e(nums: list[int]) -> int: # O(N^2) time, O(1) space
    nums.sort()  # sort then use two pointers
    for i in range(len(nums)):
        l, r = i+1, len(nums)-1 # left starts at next index, right starts at last index
        while l < r:
            if (nums[l] + nums[r] + nums[i]) > 2020:
                r -= 1
            elif (nums[l] + nums[r] + nums[i]) < 2020:
                l += 1
            elif (nums[l] + nums[r] + nums[i]) == 2020:
                return nums[l] * nums[r] * nums[i]


test_alternatives(day=1, funcs=[day1_part2, day1_part2b, day1_part2c, day1_part2d, day1_part2e], runs=10, parser=int)

Func Result:	248607374 248607374 248607374 248607374 248607374 
Func Runtime:	0.030  0.018  0.000  0.005  0.000  

## Day 02: Password Philosophy
- Input Line: #-# character: password (i.e. 1-3 e: abcde)
- Part 1: find number of passwords with character count between low and high # 
- Part 2: find number of passwords with character at idx #(1) or idx #(2) (1-based indexes and XOR)

In [7]:
def day2_parser(line: str) -> tuple:
    "Given line '1-3 e: mdie', return (1, 3, 'e', 'mdie')"
    nums, char, pw = line.split()
    a, b = map(int, nums.split('-'))
    char = char.rstrip(':')
    return (a, b, char, pw)
    

def day2_part1(policies: list[tuple]) -> int:
    cnt = 0
    for policy in policies:
        lo, hi, char, pw = policy
        if lo <= pw.count(char) <= hi:
            cnt += 1
    return cnt


def day2_part2(policies: list[tuple]) -> int:
    cnt = 0
    for policy in policies:
        pos1, pos2, char, pw = policy
        one, two = pw[pos1-1], pw[pos2-1] # minus 1 as the positions are 1-based indexed
        if (one==char) != (two==char): # could use xor (^) instead
            cnt += 1
    return cnt


run(day=2, parser=day2_parser, sep='\n', funcs=[day2_part1, day2_part2], example=True, actual=True)

Part 1 (example): 2  	Part 2 (example): 1
Part 1 (actual): 477  	Part 2 (actual): 686


In [8]:
Policy = tuple[int, int, str, str] 
Policies = list[Policy]


def day2_parserb(line: str) -> Policy:
    "Given line '1-3 e: mdie', return (1, 3, 'e', 'mdie')"
    a, b, char, pw = re.findall(r'[^-:\s]+', line) # or r'(\d+)-(\d)+ (.): (.+)$'
    return (int(a), int(b), char, pw)


def day2_part1b(policies: Policies) -> int:
    def valid_password(policy: Policy) -> bool:
        lo, hi, char, pw = policy
        return lo <= pw.count(char) <= hi
    return quantify(policies, valid_password)


def day2_part2b(policies: Policies) -> int:
    def valid_password(policy: Policy) -> bool:
        pos1, pos2, char, pw = policy
        one, two = pw[pos1-1], pw[pos2-1]  # minus 1 as these positions are 1-based indexed
        return (one==char) ^ (two==char)  # could use != instead of XOR
    return quantify(policies, valid_password)


run(day=2, parser=day2_parserb, sep='\n', funcs=[day2_part1b, day2_part2b], example=True, actual=True)

Part 1 (example): 2  	Part 2 (example): 1
Part 1 (actual): 477  	Part 2 (actual): 686


## Day 3: Toboggan Trajectory
- Input is a map of freespaces (.) & trees (#) copied infinitely to the right
- Part 1: count trees landed on moving right then down (3,1 places starting from top left)
- Part 2: return product of tree counts on the following paths: (1,1), (3,1), (5,1), (7,1), (2,1)

In [9]:
def day3_helper(treemap: list[str], x_step: int, y_step: int):
    x = 0  # starting horizontal position
    y = 0  # starting vertical position
    tree_count = 0
    
    while y < len(treemap):
        row = treemap[y]
        if row[x] == '#':  # landed on a tree
            tree_count += 1
        x = (x + x_step) % len(row)  # wrap around the x-axis
        y += y_step  # move down the y-axis
    return tree_count


def day3_part1(treemap: list[str]) -> int:
    return day3_helper(treemap, 3, 1)


def day3_part2(treemap: list[str]) -> int:
    f = lambda x, y: day3_helper(treemap, x, y)
    return f(1, 1) * f(3, 1) * f(5, 1) * f(7, 1) * f(1, 2)


run(day=3, parser=str, sep='\n', funcs=[day3_part1, day3_part2], example=True, actual=True)

Part 1 (example): 7  	Part 2 (example): 336
Part 1 (actual): 191  	Part 2 (actual): 1478615040


In [10]:
def day3_helperb(treemap: list[str], x_step=3, y_step=1) -> int:
    return quantify(row[(x_step * y) % len(row)] == '#'
                    for y, row in enumerate(treemap[::y_step]))
    # aka:
    # cnt = 0
    # for y, row in enumerate(treemap[::y_step]):
    #     # x = (x_step * y) 
    #     # pos = (x_step * y) % len(row)
    #     # print(x, y, pos)
    #     cnt += 1 if row[(x_step * y) % len(row)] == '#' else 0
    # return cnt 

def day3_part1b(treemap: list[str]) -> int:
    return day3_helperb(treemap, 3, 1)


def day3_part2b(treemap: list[str]) -> int:
    f = lambda x, y: day3_helperb(treemap, x, y)
    return f(1, 1) * f(3, 1) * f(5, 1) * f(7, 1) * f(1, 2)


run(day=3, parser=str, sep='\n', funcs=[day3_part1b, day3_part2b], example=True, actual=True)

Part 1 (example): 7  	Part 2 (example): 336
Part 1 (actual): 191  	Part 2 (actual): 1478615040


## Day 4: Passport Processing
- Input is a series of passports seperated by double newlines, each passort has # fields (key:value pairs).
- Part 1: count how many passports have required fields.
- Part 2: count how many passports have required fields and ensure each field meets set criteria.

In [11]:
REQUIRED_FIELDS = {"byr", "ecl", "eyr", "hcl", "hgt", "iyr", "pid"}


def day4_parser(line: str) -> dict:
    """Create a dict from text ala 'ecl:gry eyr:2020' -> {'ecl':'gry', 'eyr':'2020'}"""
    return dict(kv.split(":") for kv in line.split())


def day4_part1(passports: list[dict]) -> int:
    # alt.: set(p).issuperset(REQUIRED_FIELDS)
    return quantify(passports, lambda p: p.keys() >= REQUIRED_FIELDS)


def day4_part2(passports: list[dict]) -> int: 
    between = lambda v, start, end: start <= int(v) <= end
    height = lambda v: (v.endswith("cm") and between(v[:-2], 150, 193)) or (
        v.endswith("in") and between(v[:-2], 59, 76)
    )
    
    eye_colour = lambda v: v in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}
    hair_colour = lambda v: re.match("#[0-9a-f]{6}$", v)
    passport_id = lambda v: re.match("[0-9]{9}$", v)

    def valid_passport(p) -> bool:
        return p.keys() >= REQUIRED_FIELDS and all(
            (
                between(p["byr"], 1920, 2002),
                between(p["iyr"], 2010, 2020),
                between(p["eyr"], 2020, 2030),
                height(p["hgt"]),
                eye_colour(p["ecl"]),
                hair_colour(p["hcl"]),
                passport_id(p["pid"]),
            )
        )

    return quantify(passports, valid_passport)



run(day=4, parser=day4_parser, sep="\n\n", funcs=[day4_part1, day4_part2], example=True, actual=True)

Part 1 (example): 2  	Part 2 (example): 2
Part 1 (actual): 170  	Part 2 (actual): 103


## Day 5: Binary Boarding
- Input is a list of seat ids of the format 'FFFBFFF RRL' (front/back & left/right) -> 357
- Part 1: find the maximum seat id
- Part 2: find the missing seat id (in the list of contiguous seat ids)

![""](img/day5.png "day5 example")

In [12]:
def binary_partitioning(search_space: str, binary_choice):
    lb, ub = 0, (2**len(search_space))-1  # lower and upper bounds
    for item in search_space:
        if item != binary_choice:
            lb = ceil(lb/2) + ceil(ub/2)
        else:
            ub = floor(lb/2) + floor(ub/2)
    return min(lb, ub)


def day5_helper(seats: list[str]) -> list[int]:
    "Convert a list of seat strings to seat ids ala ['FBFBBFFRLR'] -> [357]."
    row = partial(binary_partitioning, binary_choice='F')
    col = partial(binary_partitioning, binary_choice='L')
    return [row(seat[:7]) * 8 + col(seat[7:]) for seat in seats]


def day5_part1(seats: list[str]) -> int:
    return max(day5_helper(seats))


def day5_part2(seats: list[int]) -> set:
    seat_ids = day5_helper(seats)
    all_seat_ids = set(i for i in range(*minmax(seat_ids)))
    return (all_seat_ids - set(seat_ids))


run(day=5, parser=str, sep='\n', funcs=[day5_part1, day5_part2], example=True, actual=True)

Part 1 (example): 357  	Part 2 (example): set()
Part 1 (actual): 888  	Part 2 (actual): {522}


In [13]:
def day5_helperb(seats: list[str], table=str.maketrans('FLBR', '0011')) -> list[int]:
    "Treat a seat description as a binary number; convert to int."
    return [int(s.translate(table), base=2) for s in seats]

def day5_part1(seats: list[str]) -> int:
    return max(day5_helperb(seats))


def day5_part2(seats: list[int]) -> set:
    seat_ids = day5_helperb(seats)
    all_seat_ids = set(i for i in range(*minmax(seat_ids)))
    return (all_seat_ids - set(seat_ids))


run(day=5, parser=str, sep='\n', funcs=[day5_part1, day5_part2], example=True, actual=True)

Part 1 (example): 357  	Part 2 (example): set()
Part 1 (actual): 888  	Part 2 (actual): {522}


## Day 6: Custom Customs
- Input is groups (seperated by \n\n) of answers (one line per person in the group).
- Part 1: Sum the count of unique answers per group.
- Part 2: Sum the count of duplicate answers per group.

In [14]:
def day6_part1(groups: list[list[str]]) -> int:
    return sum(len(set(cat(g))) for g in groups)


def day6_part2(groups: list[list[str]]) -> int:
    return sum(len(set.intersection(*map(set, g))) for g in groups)


run(day=6, parser=lines, sep='\n\n', funcs=[day6_part1, day6_part2], example=True, actual=True)

Part 1 (example): 11  	Part 2 (example): 6
Part 1 (actual): 6291  	Part 2 (actual): 3052


## Day 7: Handy Haversacks
- Input is a mapping of bag -> bags it may contain (i.e. silver -> 2 bronze bags).
- Part 1: Find the count of bags that can (eventually) contain a 'shiny gold' bag.
- Part 2: Find how many bags are contained by a 'shiny gold' bag.

In [45]:
RULE_RE = re.compile(r'^(\w+ \w+) bags contain (.+)$')
CHILD_RE = re.compile(r'(\d+) (\w+ \w+)')


def day7_part1(rules: list[str]) -> int:
    def parse(rules: list[str]) -> defaultdict[str, list[str]]:
        """Returns a {child: [parent, ...]} mapping i.e. {'shiny gold: ['bright white', ...]}"""
        child_parents = defaultdict(list)
        
        for rule in rules:
            rule_match = RULE_RE.match(rule)
            parent, rest_of_line = rule_match.groups()
            
            for num, child in CHILD_RE.findall(rest_of_line):
                child_parents[child].append(parent)
        return child_parents
    
    def find_parents(child: str, parents: defaultdict[str, list[str]]) -> set[str]:
        """Returns a set of all parents for a given child (colour), inc. itself."""
        ret = {child}
        for parent in parents[child]:
            ret |= find_parents(parent, parents)
        return ret
    
    # Recursive depth-first search 
    # i.e. using a stack (here the function call stack) to visit each leaf node
    return len(find_parents('shiny gold', parse(rules)) - {'shiny gold'})

    ## Iterative
    # child_parents = parse(rules)
    # seen = set()
    # todo = ['shiny gold']  # using list as a stack
    
    # while todo:
    #     colour = todo.pop()
    #     seen.add(colour)
    #     todo.extend(child_parents[colour])
    # return len(seen - {'shiny gold'})


def day7_part2(rules: list[str]) -> int:
    def parse(rules: list[str]) -> dict[str, tuple[int, str]]:
        "Returns a {parent: [(num, child), ...]} mapping i.e. {'shiny gold: [(1, 'Bright White'), ...]}"
        parents_can_contain = defaultdict(list)  
        
        for rule in rules:
            rule_match = RULE_RE.match(rule)
            parent, rest_of_line = rule_match.groups()
            
            children = [(int(num), child) for num, child in CHILD_RE.findall(rest_of_line)]
            parents_can_contain[parent] = children
        return parents_can_contain

    def find_bag_count(n: int, child: str, parents: dict[str, tuple[int, str]]) -> int:
        """Returns a set of all parents for a given child (colour), inc. itself."""
        ret = n
        for child_n, c in parents[child]:
            ret += find_bag_count(n * child_n, c, parents)
        return ret
    
    # Recursive depth-first search 
    return find_bag_count(1, 'shiny gold', parse(rules)) - 1


run(day=7, parser=str, sep='\n', funcs=[day7_part1, day7_part2], example=True, actual=True)

Part 1 (example): 4  	Part 2 (example): 32
Part 1 (actual): 332  	Part 2 (actual): 10875


# Day 8: Handheld Halting
- Input is a list of assembly code instructions - no operation/accumulate(to a single global var)/jump(like goto) with a int value i.e. nop +0 / jmp +4 / acc -1.
- Part 1: find the first repeated instruction (indicates an endless loop) and return the accumulated value.
- Part 2: find the buggy instruction, by replacing 1 nop with a jmp OR vice versa, the program should end at -> num-lines + 1. Return the accumulated value.


In [17]:
def day8_parser(line: str) -> tuple:
    operation, argument = line.split()
    return (operation, int(argument)) 


def day8_helper(instructions: list[tuple]) -> tuple[int, int, set[int]]:
    accumulator = current_idx = 0
    seen = set()
    
    while current_idx not in seen and current_idx != len(instructions):
        seen.add(current_idx)
        op, arg = instructions[current_idx]
        
        if op == 'acc':
            accumulator += arg
            current_idx += 1
        elif op == 'nop':
            current_idx += 1
        elif op == 'jmp':
            current_idx += arg
        else:
            AssertionError(f"Unknown instruction: {op}")
    return accumulator, current_idx, seen


def day8_part1(instructions: list[tuple]) -> int:
    return day8_helper(instructions)[0]


def day8_part2(instructions: list[tuple]) -> int:
    # skipping over instructions not used: (4.69 / 1.34) = 3.5 times faster
    possible_bad_instructions = day8_helper(instructions)[2]

    flip = {'nop': 'jmp', 'jmp': 'nop'}
    for idx, (op, arg) in enumerate(instructions):
        if idx in possible_bad_instructions:
            if op in flip.keys():
                flipped_instructions = instructions[:]
                flipped_instructions[idx] = (flip[op], arg)
                accumulator, current_idx, seen = day8_helper(flipped_instructions)
                if current_idx == len(instructions):
                    return accumulator



run(day=8, parser=day8_parser, sep='\n', funcs=[day8_part1, day8_part2], example=True, actual=True)

Part 1 (example): 5  	Part 2 (example): 8
Part 1 (actual): 1709  	Part 2 (actual): 1976


# Doing

# Day 9

- Input is a series of numbers. Each number x is proceeded by 25 (5 in example) nums, two of which sum to match x.
- Part 1: find the weird number (lacks any preceding combination of numbers that sum to match).
- Part 2: find the

In [50]:
def day9_part1(nums: list[int]) -> int: # O(n)
    def summed_combinations(nums: list[int], r: int=2) -> int:
        for pair in combinations(nums, r=2):
            yield sum(pair)
            
    skip_n = 25 if len(nums) > 20 else 5  # handle the smaller example sliding window

    for idx, num in enumerate(nums):
        if idx >= skip_n:
            # summed_pairs = {sum(pair) for pair in combinations(nums[idx:idx+25], 2)}
            summed_pairs = summed_combinations(nums[idx-skip_n:idx])
            if num not in summed_pairs:
                return num


def day9_part2(nums: list[int]) -> int:  # O(n^3)
    target = day9_part1(nums)
    for i in range(len(nums)):
        for j in range(i+1, len(nums)):
            contiguous_set = nums[i:j+1]
            if sum(contiguous_set) == target:
                return sum(minmax(contiguous_set))


run(day=9, parser=int, sep='\n', funcs=[day9_part1, day9_part2], example=True, actual=True)

Part 1 (example): 127    	Part 2 (example): 62     
Part 1 (actual): 26134589	Part 2 (actual): 3535124 


In [43]:
# 22.3 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 10 loops each) # set comp
# 18.4 ms ± 36.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) # list comp
# 4.18 ms ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) # generator function

In [55]:
def day9_part1b(nums: list[int]) -> int:
    for i in range(25, len(nums)):
        prev = nums[i-25:i]
        for x, y in combinations(prev, 2):
            if x + y == nums[i]:
                break
        else:
            return nums[i]


def day9_part1c(nums, p=25):
    """Find the first number in the list of numbers (after a preamble of p=25 numbers) 
    which is not the sum of two of the p numbers before it."""
    twosums = lambda nums: map(sum, combinations(nums, 2))
    return first(x for i, x in enumerate(nums) 
                # if i > p skips first 25, avoiding (by boolean short-circuiting) an index error on the sliced list
                if i > p and x not in twosums(nums[i-p:i]))  




test_alternatives(day=9, funcs=[day9_part1, day9_part1b, day9_part1c], runs=100, parser=int)

Func Result:	26134589  26134589  26134589  
Func Runtime:	0.195  0.112  0.114  

In [45]:
def day9_part2b(nums: list[int]) -> int:  # O(n)
    """Sliding window approach."""
    start = end = 0
    total = nums[0]
    size = len(nums)
    target = day9_part1(nums)
    
    while start < size:
        if total < target:
            end = end + 1 if end < size else size
            total += nums[end]
        elif total > target:
            total -= nums[start]
            start += 1
        else:  # total == target
            window = nums[start:end+1]
            return sum(minmax(window))


def day9_part2c(nums):
    "Find a contiguous subsequence of nums that sums to target; add their max and min."
    target = day9_part1(nums)
    subseq = find_subseq(nums, target)
    return max(subseq) + min(subseq)


def find_subseq(nums, target) -> Optional[deque]:
    "Find a contiguous subsequence of nums that sums to target."
    subseq = deque()
    total = 0
    for num in nums:
        if total < target:
            subseq.append(num)
            total += num
        if total == target and len(subseq) >= 2:
            return subseq
        while total > target:
            total -= subseq.popleft()
    return None


test_alternatives(day=9, funcs=[day9_part2, day9_part2b], runs=10, parser=int)

Func Result:	3535124  3535124  
Func Runtime:	5.897  0.020  

In [ ]:
# Learning 1
input = list(map(int, open("input/2020/input09.txt", 'r')))
n = len(input)
part1 = input[next(i for i in range(25, n) if all(input[x] + y != input[i] for x in range(i-25, i-1) for y in input[x+1:i]))]
part2 = next(x for x in (next(input[i:j] for j in range(i+1, n) if sum(input[i:j]) >= part1) for i in range(n)) if sum(x) == part1)
print(part1)
print(min(part2) + max(part2))

# js https://davidlozzi.com/2020/12/09/advent-of-code-day-9/
# https://dev.to/qviper/advent-of-code-2020-python-solution-day-9-1knj
# https://dev.to/rpalo/advent-of-code-2020-solution-megathread-day-9-encoding-error-594o

# Day 10 (review)

In [ ]:
pi = '''16
10
15
5
1
11
7
19
6
12
4
'''

pp = list(map(int, pi.splitlines()))
pp.append(0)
pp.append(max(pp) + 3)
pp.sort()
print(pp)

m = max(pp)
print(m)

#low_voltage = filter(lambda x: x <= 3, pp)
low_voltage = [x for x in pp if x <= 3]
print(low_voltage)
new = []

In [ ]:
Assume jolts[] is a vector with the input, plus 0 and max(input)+3
Assume jolts[] is sorted in ascending order
dp[0] = 1
for i = 1 to len(jolts)-1:
    dp[i] = 0
    j = i-1
    while jolts[i] - jolts[j] <= 3:
        dp[i] += dp[j]
        --j
print the last element of dp (i.e., dp[len(dp)-1])


Assume jolts[] is a vector with the input, plus 0 and max(input)+3
Assume jolts[] is sorted in ascending order
dp[0] = 1
sum = 1
j = 0
for i = 1 to len(jolts)-1:
    while jolts[i] - jolts[j] > 3:
        sum -= dp[j]
        ++j
    dp[i] = sum
    sum += dp[i]
print the last element of dp (i.e., dp[len(dp)-1])


In [ ]:

local joltages = {0} -- given input joltages; 0 is added first for the charging outlet
local diff1 = 0
local diff3 = 1 -- starts from 1 as our device's adapter has a built-in joltage of highest +3

-- read the input
for i in io.lines(input) do
    table.insert(joltages, tonumber(i))
end

-- sort the joltages
table.sort(joltages)

-- iterate over all joltages
for i = 1, (#joltages - 1) do
    -- if the difference is 1
    if joltages[i + 1] - joltages[i] == 1 then diff1 = diff1 + 1
    -- else if the difference is 3
    elseif joltages[i + 1] - joltages[i] == 3 then diff3 = diff3 + 1 end
end

print(diff1 * diff3)


In [ ]:
example = '''16
10
15
5
1
11
7
19
6
12
4
'''


joltages = list(map(int, open('/content/drive/MyDrive/advent_of_code/day10.txt', 'r')))
joltages.append(0)

diff1 = 0 # 1 jolt differences cnt
diff3 = 1 # 3 jolt differences cnt
joltages.sort()

for i in range(len(joltages)-1):
    if joltages[i+1] - joltages[i] == 1:
        diff1 = diff1 + 1
    elif joltages[i+1] - joltages[i] == 3:
        diff3 = diff3 + 1

print(diff1 * diff3)

In [ ]:
inp_file = open('/content/drive/MyDrive/advent_of_code/day10.txt')
adapters = []
for line in inp_file:
    adapters.append(int(line))

adapters.append(0)
adapters.sort()
adapters.append(adapters[-1] + 3)


class Graph:
    def __init__(self):
        self.adj_list = []

    def add_edge(self, from_vertex, to_vertex):
        self.adj_list[from_vertex].append(to_vertex)
    
    def add_vertex(self, vertex):
        self.adj_list.append(vertex)

    def __repr__(self):
        return str(self.adj_list)

    def __str__(self):
        return str(self.adj_list)


def part1():
    diffs = [0, 0, 0]

    for i in range(1, len(adapters)):
        diffs[adapters[i] - adapters[i-1] - 1] += 1

    return diffs[0] * diffs[2]



def part2():
    graph = Graph()
    current_vertex = 0
    for i in range(len(adapters)):
        new_edges = []
        for j in range(1,4):
            if i+j < len(adapters) and adapters[i+j] - adapters[i] <= 3:
                new_edges.append(current_vertex+j)

        if len(new_edges) == 1 and (len(graph.adj_list) == 0 or len(graph.adj_list[-1]) == 1):
            continue
        else:
            graph.add_vertex(new_edges)
            current_vertex += 1

    # Get the number of edges at each vertex
    num_of_edges = [len(graph.adj_list[i]) for i in range(len(graph.adj_list))]

    # Start from the second to last edge
    for i in range(len(graph.adj_list)-2, 0, -1):
        for j in range(1, 4):
            # If the current vertex is in a previous vertex
            if i-j >= 0 and i in graph.adj_list[i-j]:
                # Increase the number of edges in the previous vertex
                num_of_edges[i-j] += num_of_edges[i]
                # Remove an edge (because we're essentially doing a substitution)
                num_of_edges[i-j] -= 1
                

    return num_of_edges[0]


print (part1())
print (part2())

# Day 11

In [ ]:
import sys
from typing import Dict, Set, List, Tuple

def parse_seats(lines: List[str]) -> Dict[Tuple[int, int], Set[Tuple[int, int]]]:
    # find all seats in the input, and their connections.
    w, h = len(lines[0]), len(lines)
    seats = {(x, y): {(x, y)} for x in range(w) for y in range(h) if lines[y][x] != '.'}
    for path in generate_paths(w, h):
        previous = None
        for current in path:
            if current in seats:
                if previous:
                    seats[previous].add(current)
                    seats[current].add(previous)
                previous = current
    return seats

def generate_paths(w: int, h: int):
    # to the right
    for y in range(0, h):
        yield ((x, y) for x in range(w))
    # down
    for x in range(0, w):
        yield ((x, y) for y in range(h))
    # down and to the right
    for x0 in range(-h + 1, w):
        yield ((x0 + y, y) for y in range(max(0, -x0), min(h, w - x0)))
    # down and to the left
    for x0 in range(0, w + h):
        yield ((x0 - y, y) for y in range(max(0, x0 - w + 1), min(h, x0 + 1)))

def solve(seats: Dict[Tuple[int, int], Set[Tuple[int, int]]], n: int):
    # assume that all seats are free at the beginning.
    free = {seat for seat, visible in seats.items() if len(visible) > n} # after 2 iterations
    while True:
        # visible = {seat, *neighbours}
        changed = set(seat for seat, visible in seats.items() if visible <= free or seat not in free and len(visible - free) > n)
        if not changed:
            break
        free ^= changed
    return len(seats) - len(free)


lines = [line.rstrip() for line in open('/content/drive/MyDrive/advent_of_code/day11.txt', 'r')]
seats_part2 = parse_seats(lines)
seats_part1 = {(i, j): {(p, q) for (p, q) in visible if max(abs(i-p), abs(j-q)) < 2} for (i, j), visible in seats_part2.items()}
print(f'Part 1: {solve(seats_part1, 4)}')
print(f'Part 2: {solve(seats_part2, 5)}')

In [ ]:
from itertools import permutations

def words(letters):
    for n in range(1, len(letters)+1):
        yield from map(''.join, permutations(letters, len(letters)))


word = '12'

for i in words(word):
    print(i)

In [ ]:
x = 'asfs-afsf'
x.partition('-')

In [ ]:
class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append(end)
            else:
                self.graph_dict[start] = [end]
        print("Graph dict:", self.graph_dict)

    def get_paths(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return [path]
        
        if start not in self.graph_dict:
            return []
        
        paths = []

        for node in self.graph_dict[start]:
            if node not in path:
                new_paths = self.get_paths(node, end, path) 
                for p in new_paths:
                    paths.append(p)
        
        return paths

    def get_shortest_path(self, start, end, path=[]):
        path = path + [start]

        if start == end:
            return path

        if start not in self.graph_dict:
            return None

        shortest_path = None
        for node in self.graph_dict[start]:
            sp = self.get_shortest_path(node, end, path)
            if sp:
                if shortest_path is None or len(sp) < len(shortest_path):
                    shortest_path = sp

        return shortest_path


routes = [
          ('Mumbai', 'Paris'),
          ('Mumbai', 'Dubai'),
          ('Paris', 'Dubai'),
          ('Paris', 'New York'),
          ('Dubai', 'New York'),
          ('New York', 'Toronto')]

route_graph = Graph(routes)

start = 'Paris'
end = 'New York'

print(f'Paths between {start} and {end}: ', route_graph.get_paths(start,end))
print(f'Shortest path between {start} and {end}: ', route_graph.get_shortest_path(start,end))


In [5]:
import ast

# Python doesn't have the idea of a sign on an integer, so (neg. nums are) parsed as a unary minus:
ast.dump(ast.parse('-3').body[0])

'Expr(value=UnaryOp(op=USub(), operand=Constant(value=3)))'